In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

# Tên cuộc thi được chọn: LANL Earthquake Prediction
# Mô tả bài toán
Bài toán yêu cầu từ một dãy các kết quả dữ liệu âm thanh đầu vào, dự đoán thời gian còn lại trước khi xảy ra động đất

## Input:
Là một mảng có 150_000 dòng, chỉ gồm một giá trị là acoustic_data - dữ liệu âm thanh. Các giá trị thuộc kiểu int16.

## Output: 
Một giá trị duy nhất là thời gian còn lại trước khi xảy ra động đất. Giá trị thuộc kiểu float64.

# Mô tả dữ liệu cuộc thi
## train.csv
Là file chứa 629145481 dòng và 2 cột, là acoustic_data và time_to_failure. Các giá trị được lấy từ các thí nghiệm nghiên cứu động đất.

Acoustic_data là các dữ liệu âm thanh được lấy liên tục, còn time_to_failure là thời gian còn lại trước khi động đất tương ứng thời điểm acoustic_data được lấy.

## test
Là tệp chứa 2624 tệp để dự đoán. Mỗi tệp sẽ chứa 150_000 dòng và chỉ gồm 1 cột là acoustic_data.

## sample_submission.csv
Là file nộp mẫu.

In [ ]:
from tqdm import tqdm

In [ ]:
%%time
# Tổng số dòng dữ liệu
total = 629145481 
# Số dữ liệu được tính mỗi bước.
chunksize = 150_000

# Tải dữ liệu sử dụng chunk, do nếu tải hết dữ liệu vào trong lúc Editing Noteboong thì sẽ báo lỗi.
chunks = pd.read_csv('../input/train.csv', dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64}, chunksize = chunksize)

c = 1

segments = total // chunksize
    
# Segment là số lượng đoạn được lấy từ input.
# Các chunk được load sau đó tiền xử lý luôn luôn.

# Hiển thị đến độ chính xác 15 chữ số sau dấu phẩy.
pd.options.display.precision = 15

# Khám phá dữ liệu phân tích - Exploratory Data Analysis

Import các thư viện trực quan hóa sẽ sử dụng

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns 

plt.style.use('seaborn')

Do dữ liệu rất nhiều, nên báo cáo thực hiện nhóm mỗi 150000 dữ liệu thành một nhóm, và sử dụng max của mỗi nhóm để trực quan hóa trên biểu đồ.

Lý do sử dụng max do thực nhiệm cho thấy khi sử dụng mean, các giá trị thu được không đủ tốt để quan sát xu hướng và tính chất. Sử dụng max sẽ giúp thể hiện rõ các bất thường hơn.

In [ ]:
%%time
train = []
train_df = None

y = pd.DataFrame(index=range(segments), dtype=np.float64, columns=['time_to_failure'])

rows = 150_000

for segment in tqdm(range(segments)):
    chunk = next(chunks)
    # AD của mỗi chunk.
    ad = chunk['acoustic_data']
    # TTF của mỗi chunk. Là đầu ra cần phải dự đoán.
    y.loc[segment, 'time_to_failure'] = chunk['time_to_failure'].values[-1]
    
    # Tính các đặc trung
    features = dict()
    # Tính đặc trưng max của dữ liệu.
    features['max'] = np.max(ad)
    features['mean'] = np.mean(ad)
    # Thêm đặc trưng vào mảng train.
    train.append(features)
    
    # Chỉ lấy của 1000 segment đầu.
    if segment < 1000:
        if train_df is None:
            train_df = chunk
        else:
            train_df = train_df.append(chunk, ignore_index=True)
        del chunk
    
X = pd.DataFrame(train)

In [ ]:
plt.plot(X.index * rows, X["max"], label="acoustic_data")
plt.plot(y.index * rows, y["time_to_failure"] * 200, label="time_to_failure", color='orange')

plt.title("Biểu đồ quan hệ giữa max AD và thời gian còn lại")
plt.legend()
plt.show()

In [ ]:
plt.plot(X.index * rows, X["mean"], label="acoustic_data")
plt.plot(y.index * rows, y["time_to_failure"], label="time_to_failure", color='orange')

plt.title("Biểu đồ quan hệ giữa trung bình AD và TTF")
plt.legend()
plt.show()

In [ ]:
plt.plot(X.loc[:400].index * rows, X.loc[:400, "max"].values, label="acoustic_data")
plt.plot(y.loc[:400].index * rows, y.loc[:400, "time_to_failure"].values * 200, label="time_to_failure", color='orange')

plt.title("Biểu đồ quan hệ giữa max AD và thời gian còn lại trong 400 dòng đầu tiên")
plt.legend()
plt.show()

## Quy ước khái niệm
Báo cáo sẽ định nghĩa một số khái niệm để thuận tiện hơn trong quá trình phân tích sau này.
- Quá trình động đất - Session: là khoảng thời gian bắt đầu khi ttf tăng đột biến, đến khi ttf về gần với 0. Như biểu đồ, các quá trình động đất sẽ luôn nối tiếp nhau.
- Failure: quá trình cuối của session, khi mà ad đạt đến một giá trị rất lớn rồi giảm mạnh về rất thấp. Như trong khi biểu đồ, có nhiều vị trí mà ad tăng bất thường, nhưng không phải failure (ttf không về 0 ở các trường hợp đó)
- Loading: quá trình đầu của session, quá trình mà ad có dấu hiệu tăng dần, đến ngay trước khi Failure xảy ra.

## Phân tích biểu đồ
Quan sát các biểu đồ, báo cáo rút ra được các kết luận:

- Trước mỗi quá trình động đất mới, giá trị của ad sẽ tăng đột biến.
- Trong quá trình động đất, các giá trị của ad ở giữa cũng có xu hướng tăng dần, thỉnh thoảng sẽ có một vài giá trị cao một cách bất bình thường, tuy nhiên vẫn nhỏ hơn giá trị của acoustic data cao nhất gần vị trí cuối của quá trình động đất đó.
- Giá trị của ttf có xu hướng giảm ở cùng một tốc độ.
- Giá trị mean của ad nhỏ (<6) và không thể hiện được nhiều xu hướng.

## Kiểm tra sự thay đổi giá trị của ttf (time to failure) trong cùng một quá trình động đất

In [ ]:
plt.plot(train_df.iloc[:50000].index, train_df.iloc[:50000, 1].values)
plt.title("Biểu đồ giá trị của acoustic data trong 50_000 dòng đầu tiên")
plt.show()

In [ ]:
dt = train_df.iloc[:50000]
dt['diff'] = dt['time_to_failure'] - dt['time_to_failure'].shift(periods=1).fillna(0)

In [ ]:
dt[dt["diff"] < -0.0004]

Báo cáo nhận thấy rằng các dữ liệu thay đổi theo từng block các dòng. Mỗi block dữ liệu sẽ có 4096 dữ liệu. Trong đó sự khác biệt giữa những dữ liệu nằm trong cùng một nhóm là rất nhỏ (-0.0000000011)

Giữa 2 nhóm dữ liệu khác nhau, chênh lệch lớn hơn (-0.0009954955 và -0.0010954955), dù vậy, sự giảm thời gian khi chuyển block cũng không đều (như bảng trên). Sai khác vào khoảng 0.0001.

Theo như trong paper được cung cấp, mỗi khi xảy ra failure, stress tác dụng lên đá sẽ giảm, sao đó tăng trở lại. Khi stress đến một mức nào đó, failure sẽ xảy ra.

In [ ]:
# Tính chênh lệch giá trị giữa 2 time_to_failure liền kề.
n_block = 500
diff = len(train_df) // n_block

decay_rate = np.zeros(len(train_df))

for i in tqdm(range(n_block)):
    df = train_df.iloc[diff * i: diff * (i + 1)]
    val = df['time_to_failure'] - df['time_to_failure'].shift(periods=1).fillna(999)
    
    decay_rate[diff * i: diff * (i + 1)] = val.values

In [ ]:
# Khử đi giá trị bị Nan. 
decay_rate = decay_rate[decay_rate > -100]

In [ ]:
# Đưa về DataFrame 
decay_rate = pd.DataFrame(decay_rate, columns=["values"])
decay_rate.head()

Sự chênh lệch thường rất nhỏ, nhỏ hơn 1e-8.

In [ ]:
# Liệt kê tất cả các thời điểm xảy ra failure.

abnomal_drate = decay_rate[decay_rate['values'] > 1]
abnomal_drate

In [ ]:
# Khoảng cách giữa 2 lần xảy ra failure liên tiếp

for i in range(len(abnomal_drate.index) - 1):
    print(abnomal_drate.index[i + 1] - abnomal_drate.index[i])

Với mỗi session, chỉ có duy nhất một vị trí mà sự chênh lệch lớn hơn 1.

Số iter giữa 2 failure là rất lớn (tầm chục triệu trở lên), do đó có thể giới hạn bộ nhớ của Kaggle notebook.

### Thắc mắc: 
Ở những chỗ ngay sau failure, là lúc mà giá trị của ad rất nhỏ, nhưng cũng là lúc ttf biến động nhiều nhất, liệu với các giá trị ad có làm ảnh hưởng tới ttf giai đoạn này hay không ?

In [ ]:
plt.subplots(figsize=(30, 10))

for i in tqdm(range(len(abnomal_drate))):
    start_index = abnomal_drate.index[i] + 3
    end_index = abnomal_drate.index[i] + 150003
    
    try:
        x = train_df.iloc[start_index: end_index, 0] / 50
        y = train_df.iloc[end_index - 1, 1]

        plt.plot(range(150000), x + 4 * i, label=str(y))

        del x
        del y
    except Exception as e:
        print(e)

plt.legend()
plt.show()


#### Nhận xét biểu đồ:

Trong 150 000 iter ngay sau một Failure, ta thấy rằng những tín hiệu nào càng có nhiều biến động thì thời gian xảy ra càng lâu ? (như với tín hiệu màu tím (12.66s) hay màu xanh dương (11.5s), tuy nhiên với màu xanh dương, những tín hiệu bất thường xảy ra từ sớm, mức độ biến động của tín hiệu không cao, tuy nhiên thời gian xảy ra lại là cao nhất (14.14s).

Về cách ttf được lấy: liệu ttf được tính bằng thời gian xảy ra failure rồi trace back về iter hiện tại, hay ngay từ đầu đã xác định được ttf rồi ?, nếu nói ngay từ đầu đã tìm được ttf thì không hợp lý, do tính giảm đều ở ttf qua các iter, và việc ttf nó thế nào phụ thuộc vào thực nghiệm của failure chứ không phải tính từ đầu.

Vậy là hiểu một cách lý tính thì rất khó để từ một đoạn tính hiệu ban đầu suy ra ttf được, và nếu test case mà bao gồm các đoạn ngay sau failure thì thực sự là rất khó đoán.

Dựa vào biểu đồ ở  [link này](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2017GL074677) thì tại mỗi failure, stress sẽ bị giảm, sau đó lại tăng cho đến khi gặp một failure khác.